## Training notebook
Pandas és Polars gyakorlása lenne a cél. Adatműveletek, keresések, csoportosítások, néha átírások fognak kelleni. Polars RUST alapú. Programozósabb. Nézd meg a Pandas parancsok megfelelőit Polarsban, hogy hogy lehet használni. Most (júni 26, 2024) Pandas-ban van a legtöbb dolog, de Polars-ra szeretnénk átállni, ezért kell mindkettő.

1. chatgpt v find v vmi egy kis dataset legyen létrehozva tutorialokhoz (kaggle is jó, onnan van a manufacturing defects dataset)

2. https://www.kaggle.com/code/adilaliyev/pandas-cheatsheet-for-sql-people#Pandas-cheatsheet-for-SQL-people innen vannak kiírva a példák lentebb

3. Plus groupby dolgok, merge, like, if. The basics. Tudd gyorsan használni őket. Ezekre is van egy egy példa.

4. W3-on plot fókuszú dolgok vannak. Nem nagyon fontos de jól jöhet még.


In [1]:
#imports
import json
import pandas as pd
import numpy as np
import polars as pl

#### Reading csv, head and indexing are the same

In [2]:
manudata_pd = pd.read_csv("manufacturing_defect_dataset.csv")
manudata_pl = pl.read_csv("manufacturing_defect_dataset.csv")

In [5]:
manudata_pd.head()

,ProductionVolume,ProductionCost,SupplierQuality,DeliveryDelay,DefectRate,QualityScore,MaintenanceHours,DowntimePercentage,InventoryTurnover,StockoutRate,WorkerProductivity,SafetyIncidents,EnergyConsumption,EnergyEfficiency,AdditiveProcessTime,AdditiveMaterialCost,DefectStatus
0,202,13175.403783,86.648534,1,3.121492,63.463494,9,0.052343,8.630515,0.081322,85.042379,0,2419.616785,0.468947,5.551639,236.439301,1
1,535,19770.046093,86.310664,4,0.819531,83.697818,20,4.908328,9.296598,0.038486,99.657443,7,3915.566713,0.119485,9.080754,353.957631,1
2,960,19060.820997,82.132472,0,4.514504,90.350550,1,2.464923,5.097486,0.002887,92.819264,2,3392.385362,0.496392,6.562827,396.189402,1
3,370,5647.606037,87.335966,5,0.638524,67.628690,8,4.692476,3.577616,0.055331,96.887013,8,4652.400275,0.183125,8.097496,164.135870,1
4,206,7472.222236,81.989893,3,3.867784,82.728334,9,2.746726,6.851709,0.068047,88.315554,7,1581.630332,0.263507,6.406154,365.708964,1


In [6]:
manudata_pl.head()

ProductionVolume,ProductionCost,SupplierQuality,DeliveryDelay,DefectRate,QualityScore,MaintenanceHours,DowntimePercentage,InventoryTurnover,StockoutRate,WorkerProductivity,SafetyIncidents,EnergyConsumption,EnergyEfficiency,AdditiveProcessTime,AdditiveMaterialCost,DefectStatus
i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,i64
202,13175.403783,86.648534,1,3.121492,63.463494,9,0.052343,8.630515,0.081322,85.042379,0,2419.616785,0.468947,5.551639,236.439301,1
535,19770.046093,86.310664,4,0.819531,83.697818,20,4.908328,9.296598,0.038486,99.657443,7,3915.566713,0.119485,9.080754,353.957631,1
960,19060.820997,82.132472,0,4.514504,90.35055,1,2.464923,5.097486,0.002887,92.819264,2,3392.385362,0.496392,6.562827,396.189402,1
370,5647.606037,87.335966,5,0.638524,67.62869,8,4.692476,3.577616,0.055331,96.887013,8,4652.400275,0.183125,8.097496,164.13587,1
206,7472.222236,81.989893,3,3.867784,82.728334,9,2.746726,6.851709,0.068047,88.315554,7,1581.630332,0.263507,6.406154,365.708964,1


In [7]:
manudata_pd[0:2]

,ProductionVolume,ProductionCost,SupplierQuality,DeliveryDelay,DefectRate,QualityScore,MaintenanceHours,DowntimePercentage,InventoryTurnover,StockoutRate,WorkerProductivity,SafetyIncidents,EnergyConsumption,EnergyEfficiency,AdditiveProcessTime,AdditiveMaterialCost,DefectStatus
0,202,13175.403783,86.648534,1,3.121492,63.463494,9,0.052343,8.630515,0.081322,85.042379,0,2419.616785,0.468947,5.551639,236.439301,1
1,535,19770.046093,86.310664,4,0.819531,83.697818,20,4.908328,9.296598,0.038486,99.657443,7,3915.566713,0.119485,9.080754,353.957631,1


In [8]:
manudata_pl[0:2]

ProductionVolume,ProductionCost,SupplierQuality,DeliveryDelay,DefectRate,QualityScore,MaintenanceHours,DowntimePercentage,InventoryTurnover,StockoutRate,WorkerProductivity,SafetyIncidents,EnergyConsumption,EnergyEfficiency,AdditiveProcessTime,AdditiveMaterialCost,DefectStatus
i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,i64
202,13175.403783,86.648534,1,3.121492,63.463494,9,0.052343,8.630515,0.081322,85.042379,0,2419.616785,0.468947,5.551639,236.439301,1
535,19770.046093,86.310664,4,0.819531,83.697818,20,4.908328,9.296598,0.038486,99.657443,7,3915.566713,0.119485,9.080754,353.957631,1


#### Making a new column:
Pandas: Index non existent column and give it values. Can change the order with pop and insert.

Polars: with_columns( values.alias("colname")) the new column can overwrite the same name column,
        for index we need with_row_index()  

In [9]:
# make a new column
manudata_pd['id'] = manudata_pd.index

#reordering
# delete the column with pop
first_column = manudata_pd.pop('id')   
# insert column using insert(position,column_name, first_column) function 
manudata_pd.insert(0, 'id', first_column)

display(manudata_pd[['id','QualityScore']])

,id,QualityScore
0,0,63.463494
1,1,83.697818
2,2,90.350550
3,3,67.628690
4,4,82.728334
...,...,...
3235,3235,87.141681
3236,3236,95.562997
3237,3237,77.973442
3238,3238,95.755591


In [10]:
manudata_pl = manudata_pl.with_columns(
    pl.lit(123).alias('123')
)
manudata_pl = manudata_pl.with_row_index(name='id') #does not like to be ran multiple times

In [11]:
manudata_pl[['id','QualityScore']]

id,QualityScore
u32,f64
0,63.463494
1,83.697818
2,90.35055
3,67.62869
4,82.728334
…,…
3235,87.141681
3236,95.562997
3237,77.973442


#### Filter and replace, look for missing data:
Pandas: 
index with np.where(condition,changetovalue,atplace)
        .isnull() can be used to index NaN/None/Null...
        
Polars: 
look for and replace values with 
        with_columns( 
                pl.when( pl.col(colname)==condition )
                .then(changetovalue)
                .otherwise(pl.col(colname))
                .alias(colname)
        )
        same column name given will replace the old column automatically, we are technically adding a new column here.
        filtered display: .filter(pl.col(colname)==condition)

In [12]:
manudata_pd['QualityScore'] = np.where(manudata_pd['QualityScore'] < 64, None, manudata_pd['QualityScore'])
manudata_pd[manudata_pd['QualityScore'].isnull()][['id','QualityScore']]

,id,QualityScore
0,0,None
15,15,None
21,21,None
35,35,None
45,45,None
...,...,...
3185,3185,None
3212,3212,None
3219,3219,None
3224,3224,None


In [3]:
manudata_pl = manudata_pl.with_columns(
    (pl.when(pl.col('QualityScore')<64)
    .then(None)
    .otherwise(pl.col('QualityScore'))
    .alias('QualityScore')
    )
)
manudata_pl.filter(pl.col('QualityScore').is_null())[['id','QualityScore']]

ColumnNotFoundError: id

#### Fill nan null, select distinct/unique, sort:
Pandas: 

.fillna(tofillwith)

.drop_duplicates()

.sortvalues(bycolumn)

Polars: 

.fill_null(tofillwith)      #fill_nan for nan

.unique()

.sort(pl.col(bythiscolumn))

In [14]:
manudata_pd = manudata_pd.fillna(0)
manudata_pd['QualityScore'].drop_duplicates()
temp = manudata_pd[['id','QualityScore']].sort_values(by='QualityScore')
temp.drop_duplicates(subset='QualityScore')


/tmp/ipykernel_36714/3836028847.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  manudata_pd = manudata_pd.fillna(0)


,id,QualityScore
1110,1110,0.000000
1298,1298,64.000724
1319,1319,64.005732
2445,2445,64.060154
464,464,64.062264
...,...,...
2841,2841,99.945915
2906,2906,99.965983
1379,1379,99.978363
1204,1204,99.992206


In [15]:
manudata_pl[['id','QualityScore']].unique()
manudata_pl[['id','QualityScore']].unique().sort(pl.col('QualityScore'),nulls_last=True)

id,QualityScore
u32,f64
1298,64.000724
1319,64.005732
2445,64.060154
464,64.062264
1786,64.077276
…,…
1451,null
2574,null
2912,null


In [16]:
manudata_pl= manudata_pl.fill_null(0)
manudata_pl[['id','QualityScore']].unique(('QualityScore')).sort(pl.col('QualityScore'),nulls_last=True)


id,QualityScore
i64,f64
0,0.0
1298,64.000724
1319,64.005732
2445,64.060154
464,64.062264
…,…
2841,99.945915
2906,99.965983
1379,99.978363


#### Mean, sum, median, mode
same:

.mean()     .sum()      .median()     .mode()

In [17]:
print(f' {manudata_pd['QualityScore'].mean()} \n {manudata_pd['QualityScore'].sum()} \n {manudata_pd['QualityScore'].median()} \n {manudata_pd['QualityScore'].mode()}')

 73.37677896474875 
 237740.76384578596 
 80.26531159628432 
 0    0.0
Name: QualityScore, dtype: float64


In [18]:
print(f' {manudata_pl['QualityScore'].mean()} \n {manudata_pl['QualityScore'].sum()} \n {manudata_pl['QualityScore'].median()} \n {manudata_pl['QualityScore'].mode()}')

 73.37677896474874 
 237740.76384578593 
 80.26531159628432 
 shape: (1,)
Series: 'QualityScore' [f64]
[
	0.0
]


#### Like, or contains, starts with, ends with string (also string conversion):
same:

].str.contains('containme'), ].str.startswith('containme'), ].str.endwith('containme')

Pandas: 

].astype(str)


Polars: 

with_columns(... .cast(pl.String, strict=False))

In [19]:
temp = manudata_pd[['id','QualityScore']].astype(str) #We do this because this dataset has no string fields unfortunately
temp[temp['QualityScore'].str.startswith('6')]

,id,QualityScore
3,3,67.62869012018982
19,19,68.90119129995543
39,39,65.08816477351124
43,43,64.93022479803527
47,47,66.9764217286566
...,...,...
3191,3191,68.08337259909756
3201,3201,69.47825450352627
3208,3208,66.6176656007551
3210,3210,69.76957872352705


In [20]:
temp = manudata_pl.with_columns(pl.col("QualityScore").cast(pl.String, strict=False).alias("QualityScore"))     #we add a new column that is a cast of the old column that overwrites the old column
manudata_pl.filter(pl.col('QualityScore').str.starts_with('6'))[['id','QualityScore']] #In Polars we dont actually need to do this. str.contains, starts_with, ends_with are compatible with numbers in polars.
temp.filter(pl.col('QualityScore').str.starts_with('6'))[['id','QualityScore']] #But here you can even see the '' around the numbers

id,QualityScore
i64,str
3,"""67.62869012018982"""
19,"""68.90119129995543"""
39,"""65.08816477351124"""
43,"""64.93022479803527"""
47,"""66.9764217286566"""
…,…
3191,"""68.08337259909756"""
3201,"""69.47825450352627"""
3208,"""66.6176656007551"""


#### JOINs, similar:

Pandas:

pd.merge(table1,table2, how='left\right\inner\outer', left_on='id' , right_on='user_id')   #if the column names are the same just use  on=...

Polars:

table1.join(table2, how='left\right\inner\outer', left_on='id' , right_on='user_id')

In [21]:
t1 = manudata_pd[['id','QualityScore']]
t2 = manudata_pd[['id','DefectRate']]
pd.merge(t1,t2, how='left' , on='id')

,id,QualityScore,DefectRate
0,0,0.000000,3.121492
1,1,83.697818,0.819531
2,2,90.350550,4.514504
3,3,67.628690,0.638524
4,4,82.728334,3.867784
...,...,...,...
3235,3235,87.141681,2.667570
3236,3236,95.562997,0.751272
3237,3237,77.973442,4.899756
3238,3238,95.755591,4.057665


In [22]:
t1 = manudata_pl[['id','QualityScore']]
t2 = manudata_pl[['id','DefectRate']]
t1.join(t2,how='left',on='id')


/tmp/ipykernel_36714/2030853720.py:3: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  t1.join(t2,how='left',on='id')


id,QualityScore,DefectRate
i64,f64,f64
0,0.0,3.121492
1,83.697818,0.819531
2,90.35055,4.514504
3,67.62869,0.638524
4,82.728334,3.867784
…,…,…
3235,87.141681,2.66757
3236,95.562997,0.751272
3237,77.973442,4.899756


#### Group by:

Pandas: groupby('colname').mean()

Polars: group_by(pl.col('colname')).agg(pl.col('coltoaggregate').mean())

In [23]:
manudata_pd[['DeliveryDelay','QualityScore']].groupby('DeliveryDelay').mean().reset_index().rename(columns={"QualityScore":"Avg. QScore"})

,DeliveryDelay,Avg. QScore
0,0,73.595962
1,1,74.984327
2,2,70.399218
3,3,73.260529
4,4,73.504914
5,5,74.409951


In [24]:
manudata_pl[['DeliveryDelay','QualityScore']].group_by('DeliveryDelay').agg(pl.col('QualityScore').mean())
manudata_pl[['DeliveryDelay','QualityScore']].group_by('DeliveryDelay').agg(pl.col('QualityScore').mean()).sort(pl.col('DeliveryDelay'))    #NOT sorted by default by groupby
manudata_pl[['DeliveryDelay','QualityScore']].group_by('DeliveryDelay').agg(pl.col('QualityScore').mean()).sort(pl.col('DeliveryDelay')).rename({'QualityScore':'Avg. QScore'})

DeliveryDelay,Avg. QScore
i64,f64
0,73.595962
1,74.984327
2,70.399218
3,73.260529
4,73.504914
5,74.409951


#### Save to Jason formats for front end:

pandas: .to_dict(orient="records")
polars: .to_dicts()

Saving json is the same once the dictionary structure is achieved.

In [25]:
temp = manudata_pd[['DeliveryDelay','QualityScore','DefectRate']].groupby('DeliveryDelay').mean().reset_index().rename(columns={"QualityScore":"Avg. QScore", "DefectRate":"Avg. DRate"})
temp = temp.to_dict(orient="records")
temp

[{'DeliveryDelay': 0,
  'Avg. QScore': 73.5959618475385,
  'Avg. DRate': 2.7395657946282865},
 {'DeliveryDelay': 1,
  'Avg. QScore': 74.98432708547425,
  'Avg. DRate': 2.8047995911577477},
 {'DeliveryDelay': 2,
  'Avg. QScore': 70.39921841825486,
  'Avg. DRate': 2.84651332601182},
 {'DeliveryDelay': 3,
  'Avg. QScore': 73.26052918113986,
  'Avg. DRate': 2.720116567295347},
 {'DeliveryDelay': 4,
  'Avg. QScore': 73.50491376329146,
  'Avg. DRate': 2.6784673508922556},
 {'DeliveryDelay': 5,
  'Avg. QScore': 74.40995112027996,
  'Avg. DRate': 2.7182968180953417}]

In [26]:
with open('datagroup_pd.json', 'w') as f:
    json.dump(temp, f)

In [2]:
temp = manudata_pl[['DeliveryDelay','DefectRate','QualityScore']].group_by('DeliveryDelay').agg(pl.col('QualityScore').mean(),pl.col('DefectRate').mean()).sort(pl.col('DeliveryDelay')).rename({'QualityScore':'Avg. QScore','DefectRate':'Avg. DRate'})
temp = temp.to_dicts()
temp

NameError: name 'manudata_pl' is not defined

In [28]:
with open('datagroup_pl.json', 'w') as f:
    json.dump(temp, f)